# 기본 세팅, 데이터 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
train = pd.read_excel('/content/drive/MyDrive/와이빅타/DA_주니어플젝/데이터_2022.05.18.xlsx')
train_raw = train.copy()
train.head()

,Title,url,thumbnail
0,'과학방역' 내세운 중대본…전문가위원회·빅데이터 플랫폼 구성,https://www.yna.co.kr/view/AKR2022051803830053...,https://search.pstatic.net/common/?src=https%3...
1,"넥슨, 라프텔과 '가명정보 결합'…데이터 사이언스 강화",http://www.newsis.com/view/?id=NISX20220518_00...,https://search.pstatic.net/common/?src=https%3...
2,"인공위성 '세종 1호' 쏘는 한글과컴퓨터, 이미지 데이터 노린다",http://www.bloter.net/newsView/blt202205180027,https://search.pstatic.net/common/?src=https%3...
3,"넥슨, 라프텔과 가명정보 결합…""데이터 사이언스 강화""",https://www.news1.kr/articles/?4683961,https://search.pstatic.net/common/?src=https%3...
4,"KT SAT ""스페이스데이터 서비스 진출…글로벌 브랜드 키운다""",https://www.hankyung.com/it/article/202205184568i,https://search.pstatic.net/common/?src=https%3...


In [3]:
!pip install JPype1-1.2.0-cp38-cp38-win_amd64.whl
!pip install konlpy==0.5.2
!pip install tweepy==3.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: JPype1-1.2.0-cp38-cp38-win_amd64.whl is not a supported wheel on this platform.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 701 kB/s 
     |████████████████████████████████| 86 kB 4.7 MB/s 
     |████████████████████████████████| 453 kB 50.7 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# preprocess

In [11]:
import pandas as pd
import numpy as np
import random
import re, string
from konlpy.tag import Komoran, Hannanum
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
komoran = Komoran()
hannanum = Hannanum()


In [12]:
with open('/content/drive/MyDrive/와이빅타/DA_주니어플젝/korean_stopwords.txt', 'r', encoding='utf-8') as f:
    list_file = f.readlines() 
stopwords = list_file[0].split(",")

# 정규화, 특수기호 제거
def preprocess(text):
    text=text.strip()  
    text=re.compile('<.*?>').sub(' ', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', ' ', str(text).strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text


# 명사/영단어 추출, 한글자 제외, 불용어 제거
def final(text):
    n = []
    word = komoran.nouns(text)
    p = komoran.pos(text)
    for pos in p:
      if pos[1] in ['SL']:
        word.append(pos[0])
    for w in word:
      if len(w)>1 and w not in stopwords:
        n.append(w)
    return " ".join(n)

# 최종
def finalpreprocess(text):
  return final(preprocess(text))

In [13]:
train['noun'] = train['Title'].apply(lambda x: finalpreprocess(x))
train

,Title,url,thumbnail,noun
0,'과학방역' 내세운 중대본…전문가위원회·빅데이터 플랫폼 구성,https://www.yna.co.kr/view/AKR2022051803830053...,https://search.pstatic.net/common/?src=https%3...,과학 방역 중대 전문가 위원회 데이터 플랫폼 구성
1,"넥슨, 라프텔과 '가명정보 결합'…데이터 사이언스 강화",http://www.newsis.com/view/?id=NISX20220518_00...,https://search.pstatic.net/common/?src=https%3...,넥슨 가명 정보 결합 데이터 사이언스 강화
2,"인공위성 '세종 1호' 쏘는 한글과컴퓨터, 이미지 데이터 노린다",http://www.bloter.net/newsView/blt202205180027,https://search.pstatic.net/common/?src=https%3...,인공위성 세종 한글과컴퓨터 이미지 데이터
3,"넥슨, 라프텔과 가명정보 결합…""데이터 사이언스 강화""",https://www.news1.kr/articles/?4683961,https://search.pstatic.net/common/?src=https%3...,넥슨 가명 정보 결합 데이터 사이언스 강화
4,"KT SAT ""스페이스데이터 서비스 진출…글로벌 브랜드 키운다""",https://www.hankyung.com/it/article/202205184568i,https://search.pstatic.net/common/?src=https%3...,스페이스 데이터 서비스 진출 글로벌 브랜드 KT SAT
...,...,...,...,...
2616,﻿[2022지방선거] ﻿1400만 경기도민 표심 잡아라…경기도지사 후보 김동연 vs...,http://www.smartfn.co.kr/view.php?ud=202205181...,https://search.pstatic.net/common/?src=https%3...,지방 선거 경기도민 표심 경기도지사 후보 김동연 김은혜 vs
2617,"[사설] 가팔라질 기준금리 인상, 부채위기 확산 막아야",https://www.etoday.co.kr/news/view/2134715,,사설 기준 금리 인상 부채 위기 확산
2618,﻿[2022지방선거] 세종시장 후보 이춘희 “행정수도 완성” vs 최민호 “윤석열 ...,http://www.smartfn.co.kr/view.php?ud=202205181...,https://search.pstatic.net/common/?src=https%3...,지방 선거 세종 시장 후보 이춘희 행정 수도 완성 최민호 윤석열 정부 시작 vs
2619,"뉴욕증시 비트코인 흔들 제롬파월 연설 ""자이언트 빅스텝"" 소매판매 0.9%",http://www.g-enews.com/ko-kr/news/article/news...,https://search.pstatic.net/common/?src=https%3...,뉴욕 증시 비트코인 제롬 파월 연설 자이언트 스텝 소매 판매


# 키워드 포함 행만 추출(train_extract)

In [6]:
user_keyword = ['데이터', '머신러닝', '디지털', 'AI', '스마트', 	'DB', 'KAIST', '딥러닝', '기계학습', 'IT', '플랫폼', '블록체인', 	'핀테크', 'AWS', '구글', 'Google']

In [14]:
def extract_keyword(train):  #train은 dataframe

		# join함수를 이용하여 이어주고 contains 함수에 넣기
		user_keywords = '|'.join(user_keyword)
		temp = train[train['noun'].str.contains(user_keywords)]
		train_extract = temp[['Title', 'url', 'thumbnail']]
		train_extract
		return train_extract

# 한나눔 전처리
def final_han(text):
    n = []
    word = hannanum.nouns(text)
    p = hannanum.pos(text)
    for pos in p:
      if pos[1] in ['SL']:
        word.append(pos[0])
    for w in word:
      if len(w)>1 and w not in stopwords:
        n.append(w)
    return " ".join(n)

# 최종
def finalpreprocess_han(text):
  return final_han(preprocess(text))

In [17]:
train_extract = extract_keyword(train)
train_extract['noun'] = train_extract['Title'].apply(lambda x: finalpreprocess_han(x))
train_extract

,Title,url,thumbnail,noun
0,'과학방역' 내세운 중대본…전문가위원회·빅데이터 플랫폼 구성,https://www.yna.co.kr/view/AKR2022051803830053...,https://search.pstatic.net/common/?src=https%3...,과학방역 중대본 전문가위원회 빅데이터 플랫폼 구성
1,"넥슨, 라프텔과 '가명정보 결합'…데이터 사이언스 강화",http://www.newsis.com/view/?id=NISX20220518_00...,https://search.pstatic.net/common/?src=https%3...,넥슨 라프텔 가명정보 결합 데이터 사이언스 강화
2,"인공위성 '세종 1호' 쏘는 한글과컴퓨터, 이미지 데이터 노린다",http://www.bloter.net/newsView/blt202205180027,https://search.pstatic.net/common/?src=https%3...,인공위성 세종 한글과컴퓨터 이미지 데이터
3,"넥슨, 라프텔과 가명정보 결합…""데이터 사이언스 강화""",https://www.news1.kr/articles/?4683961,https://search.pstatic.net/common/?src=https%3...,넥슨 라프텔 가명정보 결합 데이터 사이언스 강화
4,"KT SAT ""스페이스데이터 서비스 진출…글로벌 브랜드 키운다""",https://www.hankyung.com/it/article/202205184568i,https://search.pstatic.net/common/?src=https%3...,스페이스데이터 서비스 진출 글로벌 브랜드
...,...,...,...,...
2585,"수자원공사, 제1차 디지털 혁신 릴레이포럼 개최",http://www.ikld.kr/news/articleView.html?idxno...,,수자원공사 디지털 혁신 릴레이포럼 개최
2590,한양대 임창환 교수 AI로 가상의 뇌파신호 생성기술 개발,http://www.veritas-a.com/news/articleView.html...,https://search.pstatic.net/common/?src=https%3...,한양대 임창환 교수 가상 뇌파신호 생성기술 개발
2595,"신한베트남은행, 디지털 사업 전담 추진 조직 'Future Bank Group' 출범",https://www.onews.tv/news/articleView.html?idx...,https://search.pstatic.net/common/?src=https%3...,신한베트남은행 디지털 사업 전담 추진 조직 출범
2599,한화·교보생명 오너 3세 ‘디지털’ 대결…누가 웃나,https://www.fetv.co.kr/news/article.html?no=11...,https://search.pstatic.net/common/?src=https%3...,한화 교보생명 오너 디지털 대결 누구


In [41]:
# train_extract.to_csv('/content/drive/MyDrive/와이빅타/DA_주니어플젝/final_han_preprocess.csv', index=False)

# #1 클러스터링 (같은 기사끼리 묶기)

In [19]:
text = train_extract['noun']

In [20]:
#1 tf-idf 임베딩(+Normalize)
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(min_df = 3, ngram_range=(1,5)) #min_df :최소 빈도값 (단어 등장하는 문서 수)
tfidf_vectorizer.fit(text)
vector = tfidf_vectorizer.transform(text).toarray()

vector = np.array(vector) # Normalizer를 이용해 이미 변환된 벡터



In [32]:
#2 DBSCAN Clustering
from sklearn.cluster import DBSCAN

model = DBSCAN(eps=0.1,min_samples=1, metric = "cosine") 
#     거리 계산 식으로는 Cosine distance를 이용
#     eps이 낮을수록, min_samples 값이 높을수록 군집으로 판단하는 기준이 까다로움.
result = model.fit_predict(vector)
train_extract['cluster1st'] = result

print('군집개수 :', result.max())
train_extract

군집개수 : 502


,Title,url,thumbnail,noun,cluster1st
0,'과학방역' 내세운 중대본…전문가위원회·빅데이터 플랫폼 구성,https://www.yna.co.kr/view/AKR2022051803830053...,https://search.pstatic.net/common/?src=https%3...,과학방역 중대본 전문가위원회 빅데이터 플랫폼 구성,0
1,"넥슨, 라프텔과 '가명정보 결합'…데이터 사이언스 강화",http://www.newsis.com/view/?id=NISX20220518_00...,https://search.pstatic.net/common/?src=https%3...,넥슨 라프텔 가명정보 결합 데이터 사이언스 강화,1
2,"인공위성 '세종 1호' 쏘는 한글과컴퓨터, 이미지 데이터 노린다",http://www.bloter.net/newsView/blt202205180027,https://search.pstatic.net/common/?src=https%3...,인공위성 세종 한글과컴퓨터 이미지 데이터,2
3,"넥슨, 라프텔과 가명정보 결합…""데이터 사이언스 강화""",https://www.news1.kr/articles/?4683961,https://search.pstatic.net/common/?src=https%3...,넥슨 라프텔 가명정보 결합 데이터 사이언스 강화,1
4,"KT SAT ""스페이스데이터 서비스 진출…글로벌 브랜드 키운다""",https://www.hankyung.com/it/article/202205184568i,https://search.pstatic.net/common/?src=https%3...,스페이스데이터 서비스 진출 글로벌 브랜드,3
...,...,...,...,...,...
2585,"수자원공사, 제1차 디지털 혁신 릴레이포럼 개최",http://www.ikld.kr/news/articleView.html?idxno...,,수자원공사 디지털 혁신 릴레이포럼 개최,364
2590,한양대 임창환 교수 AI로 가상의 뇌파신호 생성기술 개발,http://www.veritas-a.com/news/articleView.html...,https://search.pstatic.net/common/?src=https%3...,한양대 임창환 교수 가상 뇌파신호 생성기술 개발,398
2595,"신한베트남은행, 디지털 사업 전담 추진 조직 'Future Bank Group' 출범",https://www.onews.tv/news/articleView.html?idx...,https://search.pstatic.net/common/?src=https%3...,신한베트남은행 디지털 사업 전담 추진 조직 출범,206
2599,한화·교보생명 오너 3세 ‘디지털’ 대결…누가 웃나,https://www.fetv.co.kr/news/article.html?no=11...,https://search.pstatic.net/common/?src=https%3...,한화 교보생명 오너 디지털 대결 누구,501


In [36]:
def print_cluster_result(train):
    # 군집별 기사 개수 세는 목적
    clusters = []
    counts = []
    # 대표 기사 추출
    top_title = []
    top_noun = []
    for cluster_num in set(result):
        # -1,0은 노이즈 판별이 났거나 클러스터링이 안된 경우
        # if(cluster_num == -1 or cluster_num == 0): 
        #     continue
        # else:
            print("cluster num : {}".format(cluster_num))
            temp_df = train[train['cluster1st'] == cluster_num] # cluster num 별로 조회
            clusters.append(cluster_num)
            counts.append(len(temp_df))
            top_title.append(temp_df.reset_index()['Title'][0])
            top_noun.append(temp_df.reset_index()['noun'][0]) # 군집별 첫번째 기사를 대표기사로 ; tfidf방식
            for title in temp_df['Title']:
                print(title) # 제목으로 살펴보자
            print()

    cluster_result = pd.DataFrame({'cluster_num':clusters, 'count':counts, 'top_title':top_title, 'top_noun':top_noun})
    return cluster_result


In [37]:
cluster1_result = print_cluster_result(train_extract)
cluster1_result

cluster num : 0
'과학방역' 내세운 중대본…전문가위원회·빅데이터 플랫폼 구성
‘과학방역’ 나선 중대본, 전문가위원회·빅데이터 플랫폼 구성

cluster num : 1
넥슨, 라프텔과 '가명정보 결합'…데이터 사이언스 강화
넥슨, 라프텔과 가명정보 결합…"데이터 사이언스 강화"
넥슨, 라프텔과 가명정보 결합으로 데이터 사이언스 강화
넥슨, 라프텔과 가명정보 결합으로 데이터 사이언스 강화
넥슨, 라프텔과 가명정보 결합으로 데이터 사이언스 강화
넥슨 "라프텔과 가명정보 결합으로 데이터 사이언스 강화"

cluster num : 2
인공위성 '세종 1호' 쏘는 한글과컴퓨터, 이미지 데이터 노린다

cluster num : 3
KT SAT "스페이스데이터 서비스 진출…글로벌 브랜드 키운다"

cluster num : 4
농협은행, 마이데이터 친구추천 이벤트 진행
NH농협은행, NH마이데이터 친구추천 이벤트 진행

cluster num : 5
하동군, 공공데이터 기업매칭 공모 선정
하동군, 공공데이터 기업매칭 공모 선정
하동군, 공공데이터 기업매칭 공모 선정 [하동소식]
하동군, 공공데이터 기업매칭 공모 선정 [하동소식]
하동군, 공공데이터 기업매칭 공모 선정
하동군, 공공데이터 기업매칭 공모 선정
하동군, 공공데이터 기업매칭 공모 선정
하동군, 공공데이터 기업매칭 공모 선정
하동군, 공공데이터 기업매칭 공모 선정
하동군, 공공데이터 기업매칭 공모 선정
하동군 공공데이터 기업매칭 공모 선정

cluster num : 6
행안부, 공기관 포함 467개 기관 데이터기반행정 점검

cluster num : 7
교통사망사고 빅데이터로 도민생명 살린다
교통사망사고 빅데이터로 도민생명 살린다

cluster num : 8
“랜섬웨어 피해 기업 3곳 중 1곳, 대가 지불해도 데이터 복구 실패”
“랜섬웨어 피해 기업 3곳 중 1곳, 대가 지불해도 데이터 복구 실패”
랜섬웨어 피해 기업 76% 금전적 대가 지불... 3분의 1은 데이터 복구 못해

cluster num :

,cluster_num,count,top_title,top_noun
0,0,2,'과학방역' 내세운 중대본…전문가위원회·빅데이터 플랫폼 구성,과학방역 중대본 전문가위원회 빅데이터 플랫폼 구성
1,1,6,"넥슨, 라프텔과 '가명정보 결합'…데이터 사이언스 강화",넥슨 라프텔 가명정보 결합 데이터 사이언스 강화
2,2,1,"인공위성 '세종 1호' 쏘는 한글과컴퓨터, 이미지 데이터 노린다",인공위성 세종 한글과컴퓨터 이미지 데이터
3,3,1,"KT SAT ""스페이스데이터 서비스 진출…글로벌 브랜드 키운다""",스페이스데이터 서비스 진출 글로벌 브랜드
4,4,2,"농협은행, 마이데이터 친구추천 이벤트 진행",농협은행 마이데이터 친구추천 이벤트 진행
...,...,...,...,...
498,498,1,"SK브로드밴드, 서울디지털재단과 콘텐츠 업무 협약",SK브로드밴드 서울디지털재단 콘텐츠 업무 협약
499,499,1,"이수화학, ICT 전시회 ‘GMV 2022’서 스마트팜 사업 성과 선보인다",이수화학 전시회 스마트팜 사업
500,500,1,"신한베트남은행, 'Future Bank Group' 출범 선포...""디지털 뱅크 위...",신한베트남은행 출범 선포 디지털 뱅크 위상 강화
501,501,1,한화·교보생명 오너 3세 ‘디지털’ 대결…누가 웃나,한화 교보생명 오너 디지털 대결 누구


In [38]:
cluster1_result

,cluster_num,count,top_title,top_noun
0,0,2,'과학방역' 내세운 중대본…전문가위원회·빅데이터 플랫폼 구성,과학방역 중대본 전문가위원회 빅데이터 플랫폼 구성
1,1,6,"넥슨, 라프텔과 '가명정보 결합'…데이터 사이언스 강화",넥슨 라프텔 가명정보 결합 데이터 사이언스 강화
2,2,1,"인공위성 '세종 1호' 쏘는 한글과컴퓨터, 이미지 데이터 노린다",인공위성 세종 한글과컴퓨터 이미지 데이터
3,3,1,"KT SAT ""스페이스데이터 서비스 진출…글로벌 브랜드 키운다""",스페이스데이터 서비스 진출 글로벌 브랜드
4,4,2,"농협은행, 마이데이터 친구추천 이벤트 진행",농협은행 마이데이터 친구추천 이벤트 진행
...,...,...,...,...
498,498,1,"SK브로드밴드, 서울디지털재단과 콘텐츠 업무 협약",SK브로드밴드 서울디지털재단 콘텐츠 업무 협약
499,499,1,"이수화학, ICT 전시회 ‘GMV 2022’서 스마트팜 사업 성과 선보인다",이수화학 전시회 스마트팜 사업
500,500,1,"신한베트남은행, 'Future Bank Group' 출범 선포...""디지털 뱅크 위...",신한베트남은행 출범 선포 디지털 뱅크 위상 강화
501,501,1,한화·교보생명 오너 3세 ‘디지털’ 대결…누가 웃나,한화 교보생명 오너 디지털 대결 누구


In [40]:
# cluster1_result.to_csv("/content/drive/MyDrive/와이빅타/DA_주니어플젝/cluster1_result.csv", index=False)